# Wrangling

In [1]:
# Make the project root folder accessible
from domino.utils.jupyter import notebook_init
notebook_init()

In [2]:
import os.path
import pandas as pd
np = pd.np

import re

In [3]:
# Read the data file
players = pd.read_json(os.path.join('data', 'players.json'))

In [4]:
# Look at the first 5 rows
players.head()

,aerialWonPerGame,ap,assistTotal,goal,manOfTheMatch,minsPlayed,passSuccess,pn player_metadata,pn player_name,pn player_url,pn team_name,pn team_url,rank,rating,redCard,rgn,shotsPerGame,yellowCard
0,0.2,31(2),16,26,13,2730,81.9,"29, AM(CR),FW",Lionel Messi,https://www.whoscored.com/Players/11119,"Barcelona,",https://www.whoscored.com/Teams/65,1,8.46,-,ui-icon country flg-ar,4.8,3
1,0.6,34,12,24,7,3057,80.9,"24, AM(CLR),FW",Neymar,https://www.whoscored.com/Players/50835,"Barcelona,",https://www.whoscored.com/Teams/65,2,8.43,-,ui-icon country flg-br,3.6,6
2,0.9,21(2),10,19,5,1741,79.9,"27, M(CLR),FW",Gareth Bale,https://www.whoscored.com/Players/13812,"Real Madrid,",https://www.whoscored.com/Teams/52,3,8.12,-,ui-icon country flg-gb-wls,3.5,2
3,0.4,35,16,40,8,3150,73.1,"29, AM(CLR),FW",Luis Suárez,https://www.whoscored.com/Players/22221,"Barcelona,",https://www.whoscored.com/Teams/65,4,8.01,-,ui-icon country flg-uy,3.9,6
4,1.6,36,11,35,8,3185,79.4,"31, M(L),FW",Cristiano Ronaldo,https://www.whoscored.com/Players/5583,"Real Madrid,",https://www.whoscored.com/Teams/52,5,7.99,-,ui-icon country flg-pt,6.3,3


In [5]:
# Some columns need to be in the proper format; others need tranformation of their values
players.dtypes

aerialWonPerGame       object
ap                     object
assistTotal            object
goal                   object
manOfTheMatch          object
minsPlayed              int64
passSuccess            object
pn player_metadata     object
pn player_name         object
pn player_url          object
pn team_name           object
pn team_url            object
rank                    int64
rating                float64
redCard                object
rgn                    object
shotsPerGame           object
yellowCard             object
dtype: object

In [6]:
# For ap define a regular expression that extracts the number of appearances
players.ap = players.ap.str.extract(re.compile(r'([0-9]+)(?:\s?\(\))?'), expand=False)
players.ap = players.ap.astype(int) # when extracted the number, convert it to int

In [7]:
# Replace - for 0 and convert values to floats or ints accordingly
for col in [
    'aerialWonPerGame', 'assistTotal', 'goal', 'manOfTheMatch', 
    'minsPlayed', 'passSuccess', 'redCard', 'shotsPerGame', 'yellowCard'
]:
    players[col] = players[col].replace('-', '0')
    
    if players[col].astype(str).str.contains('.').any():
        players[col] = players[col].astype(float)
    else:
        players[col] = players[col].astype(int)

In [8]:
# rgn is the class name of a span that depicts the flag of the country the player belongs to.
# Extract with a regular expression the country code

players.rgn = players.rgn.str.extract(
    r'ui-icon country flg-((?:\w|-)+)$', 
    expand=False
).fillna('') # some rows won't have data, so fill them with ''

In [9]:
players.dtypes

aerialWonPerGame      float64
ap                      int64
assistTotal           float64
goal                  float64
manOfTheMatch         float64
minsPlayed            float64
passSuccess           float64
pn player_metadata     object
pn player_name         object
pn player_url          object
pn team_name           object
pn team_url            object
rank                    int64
rating                float64
redCard               float64
rgn                    object
shotsPerGame          float64
yellowCard            float64
dtype: object

In [10]:
players['pn player_metadata'].head()

0     29, AM(CR),FW
1    24, AM(CLR),FW
2     27, M(CLR),FW
3    29, AM(CLR),FW
4       31, M(L),FW
Name: pn player_metadata, dtype: object

In [11]:
# Extract the age of the player first
players['age'] = players['pn player_metadata'].str.extract(
    r'^([0-9]+),', 
    expand=False
).astype(int)

In [12]:
# Then extract all possible position values
players['pos'] = players['pn player_metadata'].str.findall(
    r'(?P<pos>[()A-Za-z]+)'
)

In [13]:
players.pos.head()

0     [AM(CR), FW]
1    [AM(CLR), FW]
2     [M(CLR), FW]
3    [AM(CLR), FW]
4       [M(L), FW]
Name: pos, dtype: object

In [14]:
# Create a set with all positions; sum will join all lists into one, which will be converted to a set
pos_set = set(players.pos.sum())

In [15]:
# Create pos_ columns that store booleans telling if a player plays in a given position
for pos in pos_set:
    players['pos_%s' % pos] = players.pos.apply(lambda l: pos in l)

In [16]:
players['pn team_name'].head()

0      Barcelona,
1      Barcelona,
2    Real Madrid,
3      Barcelona,
4    Real Madrid,
Name: pn team_name, dtype: object

In [17]:
# Extract team name
players['team'] = players['pn team_name'].str.extract(r'^((?:\w|\s)+)(?:,|\s)*$', expand=False)

In [18]:
# Rename some columns to a more convenient name
players = players.rename(
    columns={
        'pn player_name': 'player_name',
        'pn player_url': 'player_url',
        'pn team_url': 'team_url'
    }
)

In [19]:
# And drop useless columns
players = players.drop(
    [
        'pn player_metadata',
        'pn team_name',
        'pos'
    ], 
    axis=1
)

In [20]:
# Final result
players.head()

,aerialWonPerGame,ap,assistTotal,goal,manOfTheMatch,minsPlayed,passSuccess,player_name,player_url,team_url,...,pos_GK,pos_AM(C),pos_D(C),pos_AM(R),pos_DMC,pos_D(R),pos_AM(CL),pos_D(CLR),pos_M(CL),team
0,0.2,31,16.0,26.0,13.0,2730.0,81.9,Lionel Messi,https://www.whoscored.com/Players/11119,https://www.whoscored.com/Teams/65,...,False,False,False,False,False,False,False,False,False,Barcelona
1,0.6,34,12.0,24.0,7.0,3057.0,80.9,Neymar,https://www.whoscored.com/Players/50835,https://www.whoscored.com/Teams/65,...,False,False,False,False,False,False,False,False,False,Barcelona
2,0.9,21,10.0,19.0,5.0,1741.0,79.9,Gareth Bale,https://www.whoscored.com/Players/13812,https://www.whoscored.com/Teams/52,...,False,False,False,False,False,False,False,False,False,Real Madrid
3,0.4,35,16.0,40.0,8.0,3150.0,73.1,Luis Suárez,https://www.whoscored.com/Players/22221,https://www.whoscored.com/Teams/65,...,False,False,False,False,False,False,False,False,False,Barcelona
4,1.6,36,11.0,35.0,8.0,3185.0,79.4,Cristiano Ronaldo,https://www.whoscored.com/Players/5583,https://www.whoscored.com/Teams/52,...,False,False,False,False,False,False,False,False,False,Real Madrid


In [21]:
# Save the processed data to a CSV file
players.to_csv(os.path.join('data', 'players.csv'), index=False)